In [117]:
from neo4j import GraphDatabase, basic_auth, graph
from graphdatascience import GraphDataScience

In [42]:
password = "gcQaNwpk8pm7ADaRn-voK0CoKfibXRWprD00t_qvnMs"
user_name = "neo4j"
url = "neo4j+s://316a244c.databases.neo4j.io:7687"

In [43]:
driver = GraphDatabase.driver(url, auth=(user_name, password))

In [74]:
def find_game(game_name):
    with driver.session(database="neo4j") as session:
        result = session.execute_read(_find_and_return_game, game_name)
        for row in result:
            print("Found game: {row}".format(row=row))

def find_neighbors(node_id):
    with driver.session(database='neo4j') as session:
        result = session.execute_read(_find_neighbors, node_id)
        print(len(result))
        for row in result:
            print(row)

def collect_starting_nodes(node_id_list):
    with driver.session(database='neo4j') as session:
        result = session.execute_read(_collect_starting_nodes, node_id_list)
        return result
    



@staticmethod
def _collect_starting_nodes(tx, node_id_list):
    query = (
        """
        MATCH (n)
        WHERE n.id IN $node_id_list
        RETURN collect(n)
        """
    )
    result = tx.run(query, node_id_list=node_id_list)
    return result


@staticmethod
def _find_neighbors(tx, node_id):
    query = (
        """
        MATCH (n)-[r]-(m)
        WHERE n.id = $node_id
        RETURN collect(r)
        """
    )
    result = tx.run(query, node_id=node_id)
    return [row for row in result]


@staticmethod
def _find_and_return_game(tx, game_name):
    query = (
        "MATCH (g:Game) "
        "WHERE g.name = $game_name "
        "RETURN g"
    )
    result = tx.run(query, game_name=game_name)
    return [row for row in result]


In [45]:
find_game("Azul")

Unable to retrieve routing information
Transaction failed and will be retried in 1.0442512683029215s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 2.2328492776514546s (Unable to retrieve routing information)
Unable to retrieve routing information
Transaction failed and will be retried in 3.434040436021425s (Unable to retrieve routing information)


Found game: <Record g=<Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:69' labels=frozenset({'Game'}) properties={'short_description': 'Artfully embellish the walls of your palace by drafting the most beautiful tiles.', 'image_url': 'https://cf.geekdo-images.com/aPSHJO0d0XOpQR5X-wJonw__original/img/AkbtYVc6xXJF3c9EUrakklcclKw=/0x0/filters:format(png)/pic6973671.png', 'min_play_time': 30.0, 'expected_play_time': 45.0, 'long_description': "Introduced by the Moors, azuleijos (originally white and blue ceramic tiles) were fully embraced by the Portuguese when their king Manuel I, on a visit to the Alhambra palace in Southern Spain, was mesmerized by the stunning beauty of the Moorish decorative tiles. The king, awestruck by the interior beauty of the Alhambra, immediately ordered that his own palace in Portugal be decorated with similar wall tiles. As a tile-laying artist, you have been challenged to embellish the walls of the Royal Palace of Evora.&#10;&#10;In the game Azul, playe

In [46]:
find_neighbors("Action Queue")

1
<Record collect(r)=[<Relationship element_id='5:9c53e395-6e8b-4c0a-a50c-db53ec14015f:1521' nodes=(<Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:311' labels=frozenset() properties={}>, <Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:1050' labels=frozenset() properties={}>) type='hasMechanic' properties={'weight': 0.2}>, <Relationship element_id='5:9c53e395-6e8b-4c0a-a50c-db53ec14015f:615' nodes=(<Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:63' labels=frozenset() properties={}>, <Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:1050' labels=frozenset() properties={}>) type='hasMechanic' properties={'weight': 0.1}>, <Relationship element_id='5:9c53e395-6e8b-4c0a-a50c-db53ec14015f:1698' nodes=(<Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:371' labels=frozenset() properties={}>, <Node element_id='4:9c53e395-6e8b-4c0a-a50c-db53ec14015f:1050' labels=frozenset() properties={}>) type='hasMechanic' properties={'weight': 0.2}>, <Relationship ele

In [75]:
collect_starting_nodes(["Action Queue", "Hand Management", "Rio Grande Games"])

Particle Filtering Algorithm
1. User specifies which nodes to find (probably using dropdown selection of node types)
2. Get a collection of those starting nodes
3. Set hyperparameters for RestartProbability (c) and Threshold ($\tau$)
4. Initialize Particles (p)
5. While there are non-zero particles:
    - 

In [288]:
@staticmethod
def get_neighbors(tx, node_id):
    query = (
        """
        MATCH (n)-[r]-(m)
        WHERE n.id = $node_id
        RETURN r.weight, m.id
        """
    )
    results = tx.run(query, node_id=node_id)
    neighbors = []
    total_weight = 0.0
    for result in results:
        neighbor_id = result.get("m.id")
        relation_weight = result.get("r.weight")
        total_weight += relation_weight
        neighbors.append( (neighbor_id, relation_weight) )
    neighbors.sort(key=lambda x: x[1])
    return neighbors[-1::-1], total_weight

In [289]:
def particle_filtering(node_id_list, num_particles=10, c=0.15):
    p = {}
    v = {}
    tao = 1.0/num_particles
    with driver.session(database='neo4j') as session:

        ### Initialize Starting Nodes
        transaction = session.begin_transaction()
        query = (
        """
        MATCH (n)
        WHERE n.id IN $node_id_list
        RETURN n
        """
        )
        result = transaction.run(query, node_id_list=node_id_list)
        for node in result:
            p[node['n'].get("id")] = (1/tao)/len(node_id_list)
            v[node['n'].get("id")] = (1/tao)/len(node_id_list)

        ### Iterate through neighbors until the particles degrade to 0
        while(len(p.keys())):
            temp = {}
            for node in p.keys():
                particles = p[node] * (1-c)
                neighbors, total_weight = get_neighbors(transaction, node)
                for neighbor_id, neighbor_weight in neighbors:
                    if particles <= tao:
                        break
                    passing = particles * (neighbor_weight / total_weight)
                    if passing < tao:
                        passing = tao
                    particles -= passing
                    if neighbor_id in temp:
                        passing += temp[neighbor_id]
                    temp[neighbor_id] = passing
            p = temp
            for id, value in p.items():
                if id in v:
                    v[id] = v[id] + value
                else:
                    v[id] = value
        
        ### Filter the similar nodes for games only
        games = []
        for node_id, similarity in v.items():
            if type(node_id) == int:
                games.append((node_id, similarity))

        ### Find the information for each game
        recommendations = get_game_info(transaction, list(map(lambda x: x[0], games)))
        return recommendations


In [290]:
class Game():
    def __init__(self, attr):
        for key, value in attr.items():
            self.__setattr__(key, value)

In [295]:
recommendations = particle_filtering(["Space Exploration", "Pirates", "Theme: Cthulhu Mythos"], num_particles=10, c=.15)

<Record n.name='Pandemic: Reign of Cthulhu' n.short_description='Close portals, shut down cultists, and stave off insanity before an Old One awakens.' n.complexity_socre=2.173 n.expected_play_time=40.0 n.avg_rating=7.37 n.thumbnail_url='https://cf.geekdo-images.com/rwNaEQfzABp7dkpnwn1Ksw__micro/img/aYPNXftti9XK3v5DwpeDrS448vo=/fit-in/64x64/filters:strip_icc()/pic2866737.png' n.rank=444>
<Record n.name='Arkham Horror' n.short_description='Work as a team to save the town of Arkham from monsters and a Great Old One.' n.complexity_socre=3.5801 n.expected_play_time=240.0 n.avg_rating=7.24 n.thumbnail_url='https://cf.geekdo-images.com/9cJf4kd_HZQo6NunfJlo9w__micro/img/kOmgY2TommKpz2SMS5DjeHBqiQ8=/fit-in/64x64/filters:strip_icc()/pic175966.jpg' n.rank=397>
<Record n.name='Arkham Horror: The Card Game' n.short_description='Investigate the horrors of Arkham while courting cosmic doom.' n.complexity_socre=3.5253 n.expected_play_time=120.0 n.avg_rating=8.14 n.thumbnail_url='https://cf.geekdo-imag

In [297]:
recommendations[0].name

'Pandemic: Reign of Cthulhu'

In [293]:
games = []
for node_id, similarity in similar_nodes.items():
    if type(node_id) == int:
        games.append((node_id, similarity))

In [298]:
@staticmethod
def get_game_info(tx, game_ids):
        query = (
        """
        MATCH (n:Game)
        WHERE n.id IN $game_ids
        RETURN n.name, n.short_description, n.complexity_socre, n.expected_play_time, n.avg_rating, n.thumbnail_url, n.rank
        """
        )
        result = tx.run(query, game_ids=game_ids)

        game_recs = []
        for record in result:
            attr = {}
            attr["id"] = record.get("n.id")
            attr["num_ratings"] = record.get("n.num_ratings")
            attr["name"] = record.get("n.name")
            attr["short_descrption"] = record.get("n.short_descrption")
            attr["long_description"] = record.get("n.long_description")
            attr["complexity_score"] = record.get("n.complexity_socre")
            attr["year_published"] = record.get("n.year_published")
            attr["expected_play_time"] = record.get("n.expected_play_time")
            attr["avg_rating"] = record.get("n.avg_rating")
            attr["thumbnail_url"] = record.get("n.thumbnail_url")
            attr["image_url"] = record.get("n.image_url")
            attr["min_play_time"] = record.get("n.min_play_time")
            attr["max_play_time"] = record.get("n.max_play_time")
            attr["rank"] = record.get("n.rank")
            game_recs.append(Game(attr))
        return game_recs